In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
# change working directory to base, to make all imports and file paths work
import os
os.chdir(os.pardir)
print("Current directory: %s" % os.getcwd())

In [ ]:
import trainer.data_pipeline as dp
import trainer.constants as cst
from trainer.helpers import print_dict_keys, simple_plotly

In [ ]:
# make sure that saved_model has been created with the same version of TensorFlow
tf.__version__

In [ ]:
model_dir = "data/ion_age_20190614_205447/saved_model"
dataset_dir = cst.TEST_SET

In [ ]:
model = tf.keras.experimental.load_from_saved_model(model_dir)

# To look at available model methods:
# list(method for method in dir(model) if not method.startswith("_"))

In [ ]:
input_shapes = {k: v for k, v in zip(model.input_names, model.input_shape)}
input_shapes

In [ ]:
model.output_shape

In [ ]:
# Predict with dataset
window_size = 20
shift = 20
stride = 1
batch_size = 16

dataset = dp.create_dataset(dataset_dir,
                            window_size=window_size,
                            shift=shift,  # Can vary during validation
                            stride=stride,
                            batch_size=batch_size,  # Can vary during validation
                            cycle_length=1,  # To match original order (so no files get interleaved)
                            num_parallel_calls=1,  # Has to be equal or below cycle_length
                            shuffle=False,  # To match original order
                            repeat=False)

In [ ]:
scaling_factors = dp.load_scaling_factors()
scaling_factors

In [ ]:
predictions = []
targets = []

for i , (example, target) in enumerate(dataset):
    predictions.extend(model.predict(example).tolist())
    targets.extend(target.numpy().tolist())

predictions = np.array(predictions) * scaling_factors[cst.REMAINING_CYCLES_NAME]
targets = np.array(targets) * scaling_factors[cst.REMAINING_CYCLES_NAME]
results = pd.DataFrame({
    "pred_current_cycle": predictions[:, 0],
    "pred_remaining_cycles": predictions[:, 1],
    "target_current_cycle": targets[:, 0],
    "target_remaining_cycles": targets[:, 1],
})

In [ ]:
results["ae_current_cycle"] = (results["pred_current_cycle"] - results["target_current_cycle"]).abs()
results["ae_remaining_cycles"] = (results["pred_remaining_cycles"] - results["target_remaining_cycles"]).abs()

In [ ]:
results.head()

In [ ]:
results.hist(bins=100, figsize=(15, 10));

In [ ]:
simple_plotly(np.arange(len(results)),
              pred_remaining=results["pred_remaining_cycles"],
              target_remaining=results["target_remaining_cycles"])

# TODO:
"""
TODO
    Output relu
    Absoliute error over all cells
    Feature values over all cells and windows
"""

In [ ]:
(results["pred_remaining_cycles"] - results["target_remaining_cycles"]).abs().hist(bins=50)

In [ ]:
bins = pd.IntervalIndex.from_tuples([(-np.inf, 100), (100, 1000), (1000, np.inf)])
binning = pd.cut(results["target_remaining_cycles"], bins=bins)

binning

In [ ]:
results.hist(bins=50, figsize=(15, 10));

In [ ]:
preprocessed_pkl = dp.load_processed_battery_data()

In [ ]:
target_current_cycle = [float(k) / scaling_factors[cst.REMAINING_CYCLES_NAME] for k in preprocessed_pkl["b1c0"]["cycles"].keys()]
target_current_cycle

In [ ]:
import numpy as np

for i, (cell_k, cell_v) in enumerate(preprocessed_pkl.items()):
    if i == 1:
        break
    cycle_keys = list(cell_v["cycles"].keys())

    window_cycle_keys = []
    for i, w_slice in enumerate(range(0, len(cycle_keys), shift)):
        cycle_keys_slice = cycle_keys[w_slice:w_slice + window_size]
        if len(cycle_keys_slice) % window_size == 0:  # drop remainder
            window_cycle_keys.append(cycle_keys_slice)
    
    assert np.all([len(w) == window_size for w in window_cycle_keys]), \
        "Not all windows have the correct window_size"
    print("Number of windows in '{}': {}".format(cell_k, len(window_cycle_keys)))
    #print(window_cycle_keys)
    for j, (w_cycle_keys, example) in enumerate(zip(window_cycle_keys, dataset)):  # Iterate over all windows
        if j == 3:
            break
        print("\n##### Window {} #####".format(j))
        
        # Dataset values
        print("Example shapes:", {k:v.shape for k, v in example[0].items()})
        print("\nDataset Target:", example[1].numpy())
        # Processed Values
        processed_target = cell_v["summary"][cst.REMAINING_CYCLES_NAME] \
                                / scaling_factors[cst.REMAINING_CYCLES_NAME]
        #print("Processed Target:", processed_target[(window_size + stride * j)])
        for z, k in enumerate(w_cycle_keys):  # Itereate over all strides
            print("\n# Step")
            scaled = example[0][cst.QDLIN_NAME][:, z, :, :].numpy().squeeze() * scaling_factors[cst.QDLIN_NAME]
            print(scaled.shape)
            processed = cell_v["cycles"][k][cst.QDLIN_NAME].squeeze()
            print(processed.shape)
            print("Num close:, ", np.sum(np.isclose(scaled, processed, atol=1e-8)))
            processed_rc = (cell_v["summary"][cst.REMAINING_CYCLES_NAME][z + j * (window_size - stride)] \
                                / scaling_factors[cst.REMAINING_CYCLES_NAME]).astype(np.float32)
            print(processed_rc)
            print(cell_v["summary"].keys())

In [ ]:
class ExampleInformer:
    
    def __init__(self, processed_data_pkl, window_size, shift, stride, batch_size):
        self.processed_data_pkl = processed_data_pkl
        self.window_size = window_size,
        self.shift = shift,
        self.stride = stride,
        self.batch_size = batch_size
        self.example_info = None
    
    def _generate_example_info(self):
        self.example_info = None
    
    def get_example_info(example_number):
        return example_info
    
    def _check_if_close():
        pass
    
    def get_preprocessed_data_from_info(example_info):
        # TODO
        pass

In [ ]:
for i, (example, target) in enumerate(dataset.take(4)):
    target_cc = target.numpy()[0][0]
    preprocessed_target_cc = np.array(target_current_cycle)
    close_mask = np.isclose(target_cc, preprocessed_target_cc)
    print(target_cc)
    print(preprocessed_target_cc[close_mask][0])
    print(np.argwhere(close_mask))

In [ ]:
print_dict_keys(preprocessed_pkl["b1c1"])